<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/MyConvolutionAndDeconvolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rir_generator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import rir_generator as rir
from IPython.display import Audio 
from IPython.core.display import display
from numpy.fft import fft, ifft

#sample_rate = 512
#sample_rate = 24000
sample_rate = 2**14

def wiener_deconvolution(signal, kernel, lambd=1e-3):
  kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
  H = fft(kernel)
  Y = fft(signal)
  S = np.abs(fft(signal))**2
  #GY = Y*np.conj(H)*S/(H*np.conj(H)*S + lambd**2)
  GY = Y*np.conj(H)/(H*np.conj(H) + lambd**2)
  deconvolved = np.real(ifft(GY))
  return deconvolved

def my_conv(signal, kernel):
  kernel_padded = np.hstack((kernel, np.zeros(len(signal) - len(kernel))))
  convolved_signal = np.real(ifft(fft(kernel_padded)*fft(signal)))
  return convolved_signal
  

audiopath = '/content/drive/MyDrive/dsr_project/data/but-czas_v1.0/wavs/F-22-02/F-22-02-003.wav'
audiopath_2 = '/content/drive/MyDrive/dsr_project/data/but-czas_v1.0/wavs/M-39-10/M-39-10-001.wav'
irpath = '/content/drive/MyDrive/dsr_project/data/arthur-sykes-rymer-auditorium-university-york/b-format/s1r2.wav'

art_IR = rir.generate(
    	c=340,                  # Sound velocity (m/s)
    	fs=sample_rate,                  # Sample frequency (samples/s)
    	r=[                     # Receiver position(s) [x y z] (m)
    	    [2.5, 2, 1.5]
    	],
    	s=[3.0, 2, 1.5],          # Source position [x y z] (m)
    	L=[7, 5, 3],            # Room dimensions [x y z] (m)
    	reverberation_time=2.0, # Reverberation time T60 (s)
    	nsample=sample_rate,           # Number of output samples
	)
art_IR = art_IR[:,0]

art_IR_2 = rir.generate(
    	c=340,                  # Sound velocity (m/s)
    	fs=sample_rate,                  # Sample frequency (samples/s)
    	r=[                     # Receiver position(s) [x y z] (m)
    	    [2.5, 2, 1.5]
    	],
    	s=[3.0, 2, 1.5],          # Source position [x y z] (m)
    	L=[7, 5, 3],            # Room dimensions [x y z] (m)
    	reverberation_time=0.95*2.0, # Reverberation time T60 (s)
    	nsample=sample_rate,           # Number of output samples
	)
art_IR_2 = art_IR_2[:,0]

audio, audio_sample_rate = librosa.load(audiopath, sr=sample_rate) # Downsample to 24kHz
audio_2, audio_sample_rate_2 = librosa.load(audiopath_2, sr=sample_rate) # Downsample to 24kHz
IR, IR_sample_rate = librosa.load(irpath, sr=sample_rate) # Downsample to 24kHz

print(f"audio_sample_rate: {audio_sample_rate}, audio.shape: {audio.shape}, max: {np.max(audio)}, min: {np.min(audio)}")
print(f"IR_sample_rate:    {IR_sample_rate},    IR.shape: {IR.shape},   max: {np.max(IR)}, min: {np.min(IR)}")
display(Audio(audio[:10*sample_rate], rate=sample_rate))
display(Audio(IR, rate=sample_rate))
plt.plot(audio)
plt.show()
plt.plot(IR)
plt.show()

In [ ]:
audio_reverb = my_conv(audio, IR)
deconvolved = wiener_deconvolution(audio_reverb, IR, lambd=10**(-6))
#print(f"max: {np.max(deconvolved)}, min: {np.min(deconvolved)}")

display(Audio(audio[:2*sample_rate], rate=sample_rate))
display(Audio(audio_reverb[:2*sample_rate], rate=sample_rate))
display(Audio(deconvolved[:2*sample_rate], rate=sample_rate))

delta = audio - deconvolved
print(f"max: {np.max(delta)}, min: {np.min(delta)}, mean error: {np.sqrt(np.square(delta).mean())}")
plt.plot(delta[:2*sample_rate])
plt.show()

mes = []
for expo in range(-10, 10, 1):
  deconvolved = wiener_deconvolution(audio_reverb, IR, lambd=10**(expo))
  mes.append([10**(expo), np.sqrt(np.square(np.subtract(deconvolved, audio)).mean())])

mes

In [ ]:
audio_reverb = my_conv(audio, art_IR)
deconvolved = wiener_deconvolution(audio_reverb, art_IR, lambd=0.0)

print(f"max: {np.max(deconvolved)}, min: {np.min(deconvolved)}")
display(Audio(audio[:2*sample_rate], rate=sample_rate))
display(Audio(audio_reverb[:2*sample_rate], rate=sample_rate))
display(Audio(deconvolved[:2*sample_rate], rate=sample_rate))
delta = audio - deconvolved
print(f"max: {np.max(delta)}, min: {np.min(delta)}, mse: {np.square(delta).mean()}")
plt.plot(delta[:2*sample_rate])
plt.show()

mes = []
for expo in range(-10, 10, 1):
  deconvolved = wiener_deconvolution(audio_reverb, art_IR, lambd=10**(expo))
  mes.append([10**(expo), np.sqrt(np.square(np.subtract(deconvolved, audio)).mean())])

mes

In [ ]:
deconvolved = wiener_deconvolution(audio_reverb, art_IR_2, lambd=10**(-6))

print(f"max: {np.max(deconvolved)}, min: {np.min(deconvolved)}")
display(Audio(audio[:2*sample_rate], rate=sample_rate))
display(Audio(audio_reverb[:2*sample_rate], rate=sample_rate))
display(Audio(deconvolved[:2*sample_rate], rate=sample_rate))
delta = audio - deconvolved
print(f"max: {np.max(delta)}, min: {np.min(delta)}, mse: {np.square(delta).mean()}")
plt.plot(delta[:2*sample_rate])
plt.show()

mes = []
for expo in range(-10, 10, 1):
  deconvolved = wiener_deconvolution(audio_reverb, art_IR_2, lambd=10**(expo))
  mes.append([10**(expo), np.sqrt(np.square(np.subtract(deconvolved, audio)).mean())])

mes

In [ ]:
plt.plot(art_IR)
plt.show()
plt.plot(art_IR_2)
plt.show()

In [ ]:
delta = wiener_deconvolution(audio_reverb, art_IR_2, lambd=10**(-6)) - audio
np.square(delta).mean()

In [ ]:
from scipy.optimize import minimize

def fun(X):
  delta = wiener_deconvolution(audio_reverb, art_IR_2 + X, lambd=10**(-6)) - audio
  return np.square(delta).mean()

X0 = np.zeros(len(art_IR_2))
result = minimize(fun, X0, method='CG', options={'maxiter': 10, 'disp': True})

In [ ]:
deconvolved_2 = wiener_deconvolution(audio_reverb, art_IR_2 + result.x, lambd=10**(-6))
display(Audio(audio[:2*sample_rate], rate=sample_rate))
display(Audio(audio_reverb[:2*sample_rate], rate=sample_rate))
display(Audio(deconvolved[:2*sample_rate], rate=sample_rate))
display(Audio(deconvolved_2[:2*sample_rate], rate=sample_rate))